In [18]:
import polars as pl
import matplotlib.pyplot as plt
import pandas as pd

In [19]:
train_path='/home/ming/GraduateProject/Data/MINDlarge_train/behaviors.txt'
valid_path= '/Data/MINDlarge_dev/behaviors_1.txt'

In [20]:
train=pl.read_csv(train_path,
        separator='\t',
        has_header=False,
        new_columns=[
            'id',
            'user_id',
            'time',
            'history',
            'impressions'
        ])
valid=pl.read_csv(valid_path,
        separator='\t',
        has_header=False,
        new_columns=[
            'id',
            'user_id',
            'time',
            'history',
            'impressions'
        ])

In [21]:
train = train.with_columns(
    pl.col("time").str.strptime(
        pl.Datetime,
        format="%m/%d/%Y %I:%M:%S %p",
        strict=False
    )
)

valid = valid.with_columns(
    pl.col("time").str.strptime(
        pl.Datetime,
        format="%m/%d/%Y %I:%M:%S %p",
        strict=False
    )
)


In [22]:
valid=valid.with_columns(pl.col('history').fill_null("")
    .str.split(" ")
    .alias("history"))

In [23]:
valid=valid.with_columns(pl.col('impressions').fill_null("")
    .str.split(" ")
    .alias("impressions"))

In [24]:
valid.head()

id,user_id,time,history,impressions
i64,str,datetime[μs],list[str],list[str]
1,"""U134050""",2019-11-15 08:55:22,"[""N12246"", ""N128820"", … ""N20285""]","[""N91737-0"", ""N30206-0"", … ""N35304-0""]"
2,"""U254959""",2019-11-15 11:42:35,"[""N34011"", ""N9375"", … ""N73902""]","[""N119999-0"", ""N24958-0"", … ""N56602-0""]"
3,"""U499841""",2019-11-15 09:08:21,"[""N63858"", ""N26834"", … ""N104610""]","[""N18190-0"", ""N89764-0"", … ""N120089-0""]"
4,"""U107107""",2019-11-15 05:50:31,"[""N12959"", ""N8085"", … ""N36806""]","[""N122944-1"", ""N18190-0"", … ""N104644-0""]"
5,"""U492344""",2019-11-15 05:02:25,"[""N109183"", ""N48453"", … ""N35921""]","[""N64785-0"", ""N82503-0"", … ""N69938-0""]"


In [25]:
train.shape[0]+valid.shape[0]

2609219

In [26]:
train['user_id'].unique().count()

711222

In [27]:
valid['user_id'].unique().count()

255990

In [28]:
train['time'].min(),train['time'].max()

(datetime.datetime(2019, 11, 9, 0, 0),
 datetime.datetime(2019, 11, 14, 23, 59, 59))

In [29]:
valid['time'].min(),valid['time'].max()

(datetime.datetime(2019, 11, 15, 0, 0),
 datetime.datetime(2019, 11, 15, 23, 59, 43))

In [30]:
train.filter(pl.col('user_id')=="U239687"),valid.filter(pl.col('user_id')=="U239687")

(shape: (28, 5)
 ┌─────────┬─────────┬─────────────────────┬───────────────────────┬────────────────────────────────┐
 │ id      ┆ user_id ┆ time                ┆ history               ┆ impressions                    │
 │ ---     ┆ ---     ┆ ---                 ┆ ---                   ┆ ---                            │
 │ i64     ┆ str     ┆ datetime[μs]        ┆ str                   ┆ str                            │
 ╞═════════╪═════════╪═════════════════════╪═══════════════════════╪════════════════════════════════╡
 │ 5       ┆ U239687 ┆ 2019-11-14 20:03:01 ┆ N65250 N122359 N71723 ┆ N76209-0 N48841-0 N67937-0     │
 │         ┆         ┆                     ┆ N53796 N…             ┆ N62…                           │
 │ 29034   ┆ U239687 ┆ 2019-11-11 14:17:27 ┆ N65250 N122359 N71723 ┆ N94356-0 N107371-1 N125974-0   │
 │         ┆         ┆                     ┆ N53796 N…             ┆ N…                             │
 │ 93916   ┆ U239687 ┆ 2019-11-11 06:57:49 ┆ N65250 N122359 N71723

In [31]:
train.head()

id,user_id,time,history,impressions
i64,str,datetime[μs],str,str
1,"""U87243""",2019-11-10 11:30:54,"""N8668 N39081 N65259 N79529 N73…","""N78206-0 N26368-0 N7578-0 N585…"
2,"""U598644""",2019-11-12 13:45:29,"""N56056 N8726 N70353 N67998 N83…","""N47996-0 N82719-0 N117066-0 N8…"
3,"""U532401""",2019-11-13 11:23:03,"""N128643 N87446 N122948 N9375 N…","""N103852-0 N53474-0 N127836-0 N…"
4,"""U593596""",2019-11-12 12:24:09,"""N31043 N39592 N4104 N8223 N114…","""N38902-0 N76434-0 N71593-0 N10…"
5,"""U239687""",2019-11-14 20:03:01,"""N65250 N122359 N71723 N53796 N…","""N76209-0 N48841-0 N67937-0 N62…"


In [32]:
train=train.with_columns(pl.col('history').fill_null("")
    .str.split(" ")
    .alias("history"))

In [33]:
train=train.with_columns(pl.col('impressions').fill_null("")
    .str.split(" ")
    .alias("impressions"))

In [35]:
train.write_parquet('/home/ming/GraduateProject/Data/MINDlarge_train/behaviors_1.parquet')
valid.write_parquet('/home/ming/GraduateProject/Data/MINDlarge_dev/behaviors_1.parquet')